In [1]:
spark

# DataFrame

In [2]:
myRange = spark.range(1000).toDF("number")

위에 생성한 dataFrame은 한개의 컬럼과 1000개의 row로 구성되며 각 로우는 0~999값이 할당
이 숫자들은 분산 컬렉션을 나타내며 클러스터 모드에서 코드 예제를 실행하면 숫자범위의 각 부분이 서로 다른 익스큐터에 할당 됨.

In [3]:
myRange.show(5)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows



잘 보면 엑셀의 스프레드 시트와 비슷한데, 스파크 DataFrame은 한대의 컴퓨터가 아닌 수천대의 컴퓨터에 분산되어 저장되어 있음.

![01](./img/01/img1.png)

> __[Note]__  
> DataFrame은 스파크에서만 사용하는 개념은 아님, 파이썬과 R 모두 비슷한 개념을 가지는데, 일반적으로 분산컴퓨터가 아닌 단일 컴퓨터에 존재함.  
> 따라서, 이런경우에는 수행할 수 있는 작업이 해당 머신이 가진 자원에 따라 제한될 수밖에 없다.  
> 스파크는 파이썬,R 모두 지원하기 때문에 파이썬 pandas의 dataframe과 R의 dataframe을 스파크 Dataframe으로 쉽게 변환 가능하다.  

# 트랜스포메이션

In [8]:
divisBy2 = myRange.where("number % 2 = 0")

위 코드를 실행해도 결과는 출력되지 않는다. 추상적인 트랜스포메이션만 지정한 상태이기 때문에, 액션을 호출하지 않으면 스파크는 실제 트랜스포메이션을 수행하지 않음.

트랜스포메이션은 스파크에서 비지니스 로직을 표현하는 핵심개념이며 두가지 유형이 있다.

1) 좁은 의존성(narrow dependency)
    
    - 각 입력 파티션이 하나의 출력 파티션에만 영향을 미침
    - 위의 where 구문은 좁은 의존성

<img src="./img/02/img1.png" alt="Drawing" style="width: 400px;"/>

2) 넓은 의존성(wide dependency)
    
    - 하나의 입력 파티션이 여러 출력 파티션에 영향을 미침
    - 셔플은 다른 방식으로 동작하는데, 스파크는 셔플의 결과를 디스크에 저장하게 됨

<img src="./img/02/img2.png" alt="Drawing" style="width: 400px;"/>


> __[Note]__  
> 스파크가 클러스터에서 파티션을 교환  
> 좁은 트랜스포메이션을 사용하면 스파크에서 파이프라이닝을 자동수행  
> DataFrame에 여러 필터를 지정하는 경우 모든 작업이 메모리에서 일어난다.  


# 지연연산 (lazy evaluation)

지연 연산은 스파크가 연산 그래프를 처리하기 직전까지 기다리는 동작 방식을 의미.

1. `실행계획 생성`
    
    특정 연산 명령이 내려진 즉시 데이터를 수정하지 않고 원시 데이터에 적용할 트랜스포메이션의 <span style="color:red">[실행계획을 생성]</span>
    
    
2. `간결한 물리적 실행계획 컴파일`

    스파크는 코드를 실행하는 마지막 순간까지 대기하다가 원형 DataFrame 트랜스포메이션을 <span style="color:red">[간결한 물리적 실행 계획으로 컴파일함]</span>
   
스파크는 이 과정을 거치면서 전체 데이터 흐름을 <span style="color:red">[최적화]</span> 하는 엄청난 강점을 가지고 있음

ex) DataFrame의 조건절 푸시다운(predicate pushdown)

    아주 복잡한 스파크 잡이 원시데이터에서 하나의 로우만 가져오는 필터를 가지고 있다면, 필요한 레코드 하나만 읽는 것이 가장 효율적임.
    
    스파크는 이 필터를 데이터 소스로 위임하는 최적화 작업을 자동으로 수행한다. (어떻게..?)


# 액션

사용자는 트랜스 포메이션을 사용해 논리적 실행 계획을 세울 수 있음

하지만, 실제연산을 수행하려면 <span style="color:red">액션</span> 명령을 내려야 한다. 액션은 일련의 트랜스포메이션으로부터 결과를 계산하도록 지시하는 명령임.

가장 단순한 액션인 count 메서드는 DataFrame의 전체 레코드 수를 반환한다.

In [5]:
divisBy2.count()

500

위 코드는 500을 출력함.
이 외에, 다음 세가지 액션이 있다.
- 콘솔에서 데이터를 보는 액션
- 각 언어로 된 네이티브 객체에 데이터를 모으는 액션
- 출력 데이터소스에 저장하는 액션

액션을 지정하면 스파크 Job이 시작. 스파크 잡은 필터(좁은 트랜스포메이션)를 수행한 후, 파티션 별로 레코드 수를 카운트(넓은 트랜스포메이션)함

그 후, 각 언어에 적합한 네이티브 객체에 결과를 모음. 이 때, 스파크가 제공하는 스파크 UI로 클러스터에서 실행 중인 스파크 잡을 모니터링 할 수 있다.

In [7]:
divisBy2.show(5)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
+------+
only showing top 5 rows



# 스파크UI
스파크UI는 스파크 잡의 진행상황을 모니터링 할 때 사용

스파크 UI는 드라이버 노드의 4040포트로 접속 할 수 있다. 로컬에서 스파크를 실행 했다면, 스파크 UI주소는 <a href =http://localhost:4040>http://localhost:4040</a> 이 된다. (링크를 클릭하면 다음과 같은 화면이 나옴)

<img src=./img/02/img3.png>

스파크 UI
- 스파크 잡의 상태
- 환경설정
- 클러스터 상태 등의 정보를 확인할 수 있음.

스파크 잡을 튜닝하고 디버깅할 때 매우 유용하다.

스파크 잡은 개별 액션에 의해 트리거되는 `다수의 트랜스포메이션`으로 이뤄져 있으며, 스파크 UI로 잡을 모니터링 할 수있음.